In [11]:
import pandas as pd


payments = pd.read_csv("data/home_credit/installments_payments.csv")
cash_position = pd.read_csv("data/home_credit/POS_CASH_balance.csv")

In [12]:
# Inspecting the data
payments.head(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
5,1137312,164489,1.0,12,-1384.0,-1417.0,5970.375,5970.375
6,2234264,184693,4.0,11,-349.0,-352.0,29432.295,29432.295
7,1818599,111420,2.0,4,-968.0,-994.0,17862.165,17862.165
8,2723183,112102,0.0,14,-197.0,-197.0,70.740,70.740
9,1413990,109741,1.0,4,-570.0,-609.0,14308.470,14308.470


In [13]:
cash_position.head(10)
cash_position.NAME_CONTRACT_STATUS.unique()

array(['Active', 'Completed', 'Signed', 'Approved',
       'Returned to the store', 'Demand', 'Canceled', 'XNA',
       'Amortized debt'], dtype=object)

In [14]:
# Observing the behavior of a single previous loan
data_single_loan = payments[payments['SK_ID_PREV'] == 2714724]
data_single_loan.sort_values(by=['NUM_INSTALMENT_NUMBER'], inplace=True)
data_single_loan["payments_due_days"] = (data_single_loan.DAYS_ENTRY_PAYMENT - data_single_loan.DAYS_INSTALMENT)

print(data_single_loan)

         SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  \
2843935     2714724      167756                     1.0   
4           2714724      167756                     1.0   
1786772     2714724      167756                     1.0   
422906      2714724      167756                     1.0   
2998490     2714724      167756                     1.0   
2201494     2714724      167756                     2.0   

         NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  \
2843935                      1          -1413.0             -1421.0   
4                            2          -1383.0             -1366.0   
1786772                      2          -1383.0             -1421.0   
422906                       3          -1353.0             -1366.0   
2998490                      3          -1353.0             -1336.0   
2201494                      4          -1323.0             -1336.0   

         AMT_INSTALMENT  AMT_PAYMENT  payments_due_days  
2843935         2165.04     21

<ipython-input-14-f1b7d74ca553>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_single_loan.sort_values(by=['NUM_INSTALMENT_NUMBER'], inplace=True)
<ipython-input-14-f1b7d74ca553>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_single_loan["payments_due_days"] = (data_single_loan.DAYS_ENTRY_PAYMENT - data_single_loan.DAYS_INSTALMENT)


In [15]:
cash_position_single_loan = cash_position[cash_position['SK_ID_PREV'] == 2714724]
cash_position_single_loan.sort_values(by=['CNT_INSTALMENT_FUTURE'], ascending=False, inplace=True)

print(cash_position_single_loan)

         SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
7673576     2714724      167756             -47             8.0   
8660962     2714724      167756             -46             8.0   
1860851     2714724      167756             -45             8.0   
2753540     2714724      167756             -44             8.0   
686245      2714724      167756             -43             4.0   

         CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
7673576                    4.0               Active       0           0  
8660962                    3.0               Active       0           0  
1860851                    2.0               Active      15          15  
2753540                    1.0               Active      16          16  
686245                     0.0            Completed       0           0  


<ipython-input-15-598bc0cda353>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_position_single_loan.sort_values(by=['CNT_INSTALMENT_FUTURE'], ascending=False, inplace=True)


In [16]:
contracts_in_default = cash_position.query("SK_DPD >= 30")

print(f'Total number of contracts in default area {contracts_in_default.SK_ID_PREV.nunique()}')

Total number of contracts in default area 8678


In [17]:
contracts_canceled = cash_position.query("NAME_CONTRACT_STATUS == 'Canceled'")

print(f'Total number of contracts in default area {contracts_canceled.SK_ID_PREV.nunique()}')

contracts_canceled.head(20)

Total number of contracts in default area 15


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
972025,2200360,256843,-6,NaN,NaN,Canceled,0,0
3804943,2762733,307509,-37,12.0,12.0,Canceled,0,0
4727178,1720362,387292,-19,NaN,NaN,Canceled,0,0
5018718,1531600,103793,-28,NaN,NaN,Canceled,0,0
7348078,2239989,347381,-2,NaN,NaN,Canceled,0,0
7556193,2134263,366250,-8,NaN,NaN,Canceled,0,0
7617341,2122901,390665,-3,NaN,NaN,Canceled,0,0
8255924,1287451,408925,-14,NaN,NaN,Canceled,0,0
8284174,1179494,186302,-57,11.0,11.0,Canceled,0,0
8433676,1176218,253007,-23,NaN,NaN,Canceled,0,0


In [18]:
# Build the sequence of payments days and the next payment installment situation

# Para cada SK_ID_PREV:
### ordenar os pagamentos
### montar a sequencia de pagamentos ate t-1 e o resultado em t
### e.g: (para SK_ID_PREV == 1054186)
#### [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0] 5 OR On-time
#### [8, 7, 5, 0, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 0, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 7, 0, 0, 0, 0, 0, 0] 7 OR On-time
#### [8, 7, 5, 7, 7, 7, 0, 0, 0, 0, 0] 25 OR On-time
#### [8, 7, 5, 7, 7, 7, 25, 0, 0, 0, 0] 55 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 0, 0, 0] 44 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 0, 0] 35 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 0] 33 OR Late
#### [8, 7, 5, 7, 7, 7, 25, 55, 44, 35, 33] 33 OR Late

In [19]:
# Function to calculate payment status
def payment_status(days_between_payment_due_date):
    if days_between_payment_due_date < -30:
        return "Advance payment"
    if -30 < days_between_payment_due_date <= 0:
        return "Payment on schedule"
    if 0 < days_between_payment_due_date <= 30:
        return "Late payment within 30 days"
    if 30 < days_between_payment_due_date <= 90:
        return "Late payment 30+ days but no default"
    if days_between_payment_due_date > 90:
        return "Default"

In [44]:
loan_payments = cash_position[cash_position.SK_ID_PREV == 2714724]
loan_payments.sort_values(by=['CNT_INSTALMENT_FUTURE'], ascending=False, inplace=True)
loan_payments = loan_payments[["CNT_INSTALMENT_FUTURE", "SK_DPD"]]
loan_payments["PAYMENT_STATUS"] = loan_payments["SK_DPD"].apply(lambda x: payment_status(x))  

loan_payments

<ipython-input-44-d9f6c95ef3e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_payments.sort_values(by=['CNT_INSTALMENT_FUTURE'], ascending=False, inplace=True)


,CNT_INSTALMENT_FUTURE,SK_DPD,PAYMENT_STATUS
7673576,4.0,0,Payment on schedule
8660962,3.0,0,Payment on schedule
1860851,2.0,15,Late payment within 30 days
2753540,1.0,16,Late payment within 30 days
686245,0.0,0,Payment on schedule


In [45]:
loan_payments = loan_payments[1:]
num_payments = len(loan_payments.SK_DPD)

payments_behavior = [loan_payments.SK_DPD[0:i].tolist() + [0] * (num_payments - i - 1) for i in range(0, num_payments)][1:]

next_period_status = loan_payments.PAYMENT_STATUS[1:]

payments_historical_behavior = {
    'payments_history': payments_behavior,
    'next_period_status': next_period_status
}

payments_history = pd.DataFrame(payments_historical_behavior)
payments_history

,payments_history,next_period_status
1860851,"[0, 0, 0]",Late payment within 30 days
2753540,"[0, 15, 0]",Late payment within 30 days
686245,"[0, 15, 16]",Payment on schedule


In [36]:

# Write as a generator
def get_payments_history(cash_position_df, sk_id_prev):
    
    loan_payments = cash_position_df[cash_position_df.SK_ID_PREV == sk_id_prev]
    loan_payments.sort_values(by=['CNT_INSTALMENT_FUTURE'], ascending=False, inplace=True)
    loan_payments = loan_payments[["CNT_INSTALMENT_FUTURE", "SK_DPD"]]
    loan_payments["PAYMENT_STATUS"] = loan_payments["SK_DPD"].apply(lambda x: payment_status(x))
    
    loan_payments = loan_payments[1:]
    
    payments_behavior = [loan_payments.SK_DPD[0:i].tolist() for i in range(0, len(loan_payments.SK_DPD))][1:]

    next_period_status = loan_payments.PAYMENT_STATUS[1:]

    payments_historical_behavior = {
        'payments_history': payments_behavior,
        'next_period_status': next_period_status
    }

    payments_history = pd.DataFrame(payments_historical_behavior)
    
    return payments_history

    

In [39]:
completed_contracts = cash_position.query("NAME_CONTRACT_STATUS == 'Completed'")['SK_ID_PREV'].unique().tolist()

final_df = pd.DataFrame()
for contract in completed_contracts:
    payment_history = get_payment_history(cash_position, contract)
    # Append to Final DF

[1298869,
 1955907,
 2714240,
 1655882,
 2567983,
 1192377,
 1752908,
 2612717,
 1921577,
 1102362,
 1329480,
 2566846,
 1464874,
 1840591,
 1622838,
 1204721,
 2373929,
 2116637,
 2789273,
 1241675,
 1692671,
 2605651,
 2315839,
 2446077,
 1012853,
 2762932,
 1034137,
 1979139,
 2166314,
 1230285,
 1869710,
 1009382,
 1652748,
 2437940,
 1118673,
 1783406,
 1649154,
 2246145,
 1643886,
 2575938,
 2243927,
 1856089,
 1765643,
 2581933,
 2769176,
 1882131,
 2366181,
 1027217,
 1772994,
 1746090,
 1873302,
 1915742,
 1849660,
 1267994,
 1226607,
 2836622,
 1305067,
 2198660,
 2497013,
 1995586,
 1818295,
 1816521,
 1636418,
 1847412,
 1548079,
 1770040,
 2166947,
 1932088,
 1176324,
 1271632,
 1002667,
 1335160,
 2404751,
 1096155,
 2064689,
 2608370,
 2504969,
 2080203,
 1794118,
 2628971,
 1756690,
 2305964,
 1133579,
 2367058,
 1837079,
 2029867,
 1540056,
 1575153,
 1513175,
 1455705,
 2706182,
 1029405,
 1493655,
 2497094,
 2415095,
 1364429,
 2056984,
 1027630,
 2618109,
 2746523,
